In [1]:
import trimesh
from tqdm import tqdm
import utils.geometry_util as geometry_util
import utils.shape_util as shape_util
import numpy as np
import os

scene = trimesh.Scene()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from my_code.datasets.surreal_dataset_3dc import TemplateSurrealDataset3DC

user_name = 's94zalek_hpc'
dataset = TemplateSurrealDataset3DC(
    shape_path=f'/home/{user_name}/3D-CODED/data/datas_surreal_train.pth',
    num_evecs=4,
    use_cuda=False,
    cache_lb_dir=None,
    return_evecs=False
)    

FileNotFoundError: [Errno 2] No such file or directory: '/home/s94zalek_hpc/3D-CODED/data/datas_surreal_train.pth'

In [3]:
base_folder = '/home/s94zalek_hpc/shape_matching/data_sign_training/train/SURREAL'


In [18]:
# base_folder = '/home/s94zalek_hpc/shape_matching/data_sign_training/train/SURREAL'


base_folder = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/data_sign_training/train/SURREAL'

mesh_folder = f'{base_folder}/off'
corr_folder = f'{base_folder}/corres'

os.makedirs(mesh_folder)
os.makedirs(corr_folder)

n_shapes = 1000
rand_idxs = np.random.choice(len(dataset), n_shapes, replace=False)

for curr_iter, i in tqdm(enumerate(rand_idxs), total=n_shapes):
    data = dataset[i]['second']
    

    shape_util.write_off(
        f'{mesh_folder}/{curr_iter:04}.off',
        data['verts'].numpy(),
        data['faces'].numpy()
        )
    
    with open(f'{corr_folder}/{curr_iter:04}.vts', 'w') as f:
        corr = np.array(list(range(len(data['verts'])))) + 1
        np.savetxt(f, corr, fmt='%d')
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:08<00:00,  7.75it/s]


In [37]:
!printenv HOME

/home/s94zalek_hpc


In [40]:
!ls /tmp

In [ ]:
scene.geometry.clear()

mesh_folder = '/home/s94zalek_hpc/shape_matching/data_sign_training/train/SURREAL_train_rot_180_180_180_normal_True_noise_0.0_-0.05_0.05_lapl_mesh_scale_0.9_1.1/off'
# read 5 random shapes and show them
rand_idxs = np.random.choice(1000, 5, replace=False)

for curr_idx, i in enumerate(rand_idxs):
    mesh = trimesh.load_mesh(f'{mesh_folder}/{i:04}.off')
    mesh.vertices += np.array([curr_idx, 0, 0])
    scene.add_geometry(mesh)
    
scene.show()

In [4]:
import my_code.datasets.shape_dataset as shape_dataset

diff_folder = f'{base_folder}/diffusion'
dataset_single = shape_dataset.SingleShapeDataset(
    data_root = base_folder,
    centering = 'bbox',
    num_evecs=128,
    lb_cache_dir=diff_folder,
)  

In [5]:
data_train = []
for i in tqdm(range(len(dataset_single))):
    data_i = dataset_single[i]
    data_train.append({
        'verts': data_i['verts'],
        'faces': data_i['faces'],
        'evecs': data_i['evecs'],
        })

 21%|███████████████████▉                                                                           | 210/1000 [00:46<02:54,  4.53it/s]


KeyboardInterrupt: 